In [1]:
# %load_ext memory_profiler
!pip install -q zhconv

In [2]:
!pip install gensim

In [3]:
!pip install wget

In [4]:
import urllib.request
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
filename = "dict.txt.big"
urllib.request.urlretrieve(url, filename)

('dict.txt.big', <http.client.HTTPMessage at 0x22907589cf0>)

In [5]:
!pip install jieba

In [6]:
import os 

# Packages
import gensim
import jieba
import zhconv
from gensim.corpora import WikiCorpus
from datetime import datetime as dt
from typing import List


if not os.path.isfile('dict.txt.big'):
    !wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
jieba.set_dictionary('dict.txt.big')

print("gensim", gensim.__version__)
print("jieba", jieba.__version__)

gensim 4.3.0
jieba 0.42.1


In [7]:
ZhWiki = r'C:\Users\User\Downloads\zhwiki-20230501-pages-articles-multistream.xml.bz2'

In [8]:
zhconv.convert("这原本是一段简体中文", "zh-tw")

'這原本是一段簡體中文'

In [9]:
print(list(jieba.cut("中英夾雜的example，Word2Vec應該很interesting吧?")))

Building prefix dict from C:\Users\User\Downloads\dict.txt.big ...
Dumping model to file cache C:\Users\User\AppData\Local\Temp\jieba.u9f3f7311224433a41d53eef275394fc7.cache
Loading model cost 1.190 seconds.
Prefix dict has been built successfully.


['中', '英', '夾雜', '的', 'example', '，', 'Word2Vec', '應該', '很', 'interesting', '吧', '?']


In [10]:
!pip install spacy

In [11]:
import spacy

# 下載語言模組
spacy.cli.download("zh_core_web_sm")  # 下載 spacy 中文模組
spacy.cli.download("en_core_web_sm")  # 下載 spacy 英文模組

nlp_zh = spacy.load("zh_core_web_sm") # 載入 spacy 中文模組
nlp_en = spacy.load("en_core_web_sm") # 載入 spacy 英文模組

# 印出前20個停用詞
print('--\n')
print(f"中文停用詞 Total={len(nlp_zh.Defaults.stop_words)}: {list(nlp_zh.Defaults.stop_words)[:20]} ...")
print("--")
print(f"英文停用詞 Total={len(nlp_en.Defaults.stop_words)}: {list(nlp_en.Defaults.stop_words)[:20]} ...")

✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
--

中文停用詞 Total=1891: ['最大', '一来', '日渐', '挨个', '从古到今', '尽管如此', '无论', '哎呀', '不惟', '方才', '着', '大概', '等到', '屡屡', '挨家挨户', '挨次', '/', '（', '连连', '忽地'] ...
--
英文停用詞 Total=326: ['perhaps', 'take', 'never', 'meanwhile', 'due', 'after', 'hers', 'those', 'could', 'keep', 'around', 'herein', 'whither', 'from', 'behind', 'into', 'alone', 'now', 'everything', 'other'] ...


In [12]:
STOPWORDS =  nlp_zh.Defaults.stop_words | \
             nlp_en.Defaults.stop_words | \
             set(["\n", "\r\n", "\t", " ", ""])
print(len(STOPWORDS))

# 將簡體停用詞轉成繁體，擴充停用詞表
for word in STOPWORDS.copy():
    STOPWORDS.add(zhconv.convert(word, "zh-tw"))
    
print(len(STOPWORDS))

2222
3005


In [13]:
def preprocess_and_tokenize(
    text: str, token_min_len: int=1, token_max_len: int=15, lower: bool=True) -> List[str]:
    if lower:
        text  = text.lower()
    text = zhconv.convert(text, "zh-tw")
    return [
        token for token in jieba.cut(text, cut_all=False)
        if token_min_len <= len(token) <= token_max_len and \
            token not in STOPWORDS
    ]

In [14]:
print(preprocess_and_tokenize("歐幾里得，西元前三世紀的古希臘數學家，現在被認為是幾何之父，此畫為拉斐爾"))
print(preprocess_and_tokenize("我来到北京清华大学"))
print(preprocess_and_tokenize("中英夾雜的example，Word2Vec應該很interesting吧?"))

['歐幾', '裡得', '西元前', '世紀', '古希臘', '數學家', '幾何', '父', '此畫', '拉斐爾']
['來到', '北京', '清華大學']
['中', '英', '夾雜', 'example', 'word2vec', 'interesting']


In [15]:
print(f"Parsing {ZhWiki}...")
wiki_corpus = WikiCorpus(ZhWiki, token_min_len=1)

Parsing C:\Users\User\Downloads\zhwiki-20230501-pages-articles-multistream.xml.bz2...


C:\Users\User\anaconda3\lib\site-packages\gensim\utils.py:1333: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)


In [16]:
g = wiki_corpus.get_texts()
print(next(g)[:10])
print(next(g)[:10])
print(next(g)[:10])


# print(jieba.lcut("".join(next(g))[:50]))
# print(jieba.lcut("".join(next(g))[:50]))

['歐幾里得', '西元前三世紀的古希臘數學家', '現在被認為是幾何之父', '此畫為拉斐爾的作品', '雅典學院', '数学', '是研究數量', '屬於形式科學的一種', '數學利用抽象化和邏輯推理', '從計數']
['蘇格拉底之死', '由雅克', '路易', '大卫所繪', '年', '哲學', '是研究普遍的', '基本问题的学科', '包括存在', '知识']
['文學', '在狭义上', '是一种语言艺术', '亦即使用语言文字为手段', '形象化地反映客观社会生活', '表达主观作者思想感情的一种艺术', '文学不仅强调传达思想观念', '更强调传达方式的独特性', '且讲究辞章的美感', '文学']


In [4]:
WIKI_SEG_TXT = "wiki_seg.txt"

In [5]:
%%time

from gensim.models import word2vec
import multiprocessing

max_cpu_counts = multiprocessing.cpu_count()
word_dim_size = 300  #  設定 word vector 維度
print(f"Use {max_cpu_counts} workers to train Word2Vec (dim={word_dim_size})")


# 讀取訓練語句
sentences = word2vec.LineSentence(WIKI_SEG_TXT)

# 訓練模型
model = word2vec.Word2Vec(sentences, vector_size=word_dim_size, workers=max_cpu_counts)

# 儲存模型
output_model = f"word2vec.zh.{word_dim_size}.model"
model.save(output_model)

Use 12 workers to train Word2Vec (dim=300)
CPU times: total: 1h 9min 37s
Wall time: 29min 4s


In [6]:
print(model.wv.vectors.shape)
model.wv.vectors

(1281108, 300)


array([[ 2.8076830e+00,  1.2115554e+00, -3.0168431e+00, ...,
         2.1298341e-01,  1.0413009e+00, -1.3318332e+00],
       [ 2.3509305e+00, -7.0053291e-01, -2.4598410e+00, ...,
         8.5789061e-01,  2.8712637e+00, -3.0379291e+00],
       [ 9.7158843e-01, -3.6645389e-01, -1.2535313e+00, ...,
        -2.2477122e-02,  6.4034611e-03, -1.3746341e-01],
       ...,
       [ 6.7367656e-03,  4.7338571e-02,  9.2347644e-02, ...,
        -2.3707920e-03, -5.1538050e-02, -8.7550983e-02],
       [-1.8512698e-02, -1.0023722e-02, -1.1426814e-02, ...,
        -5.8442885e-05,  2.7651146e-02, -9.6827056e-03],
       [-5.0373469e-02,  2.7495909e-02,  3.9573699e-02, ...,
         1.7450697e-03, -5.5091362e-02, -2.2538140e-02]], dtype=float32)

In [7]:
print(f"總共收錄了 {len(model.wv.key_to_index)} 個詞彙")

print("印出 20 個收錄詞彙:")
print(list(model.wv.key_to_index.keys())[:10])


總共收錄了 1281108 個詞彙
印出 20 個收錄詞彙:
['年', '月', '日', '中', '10', '12', '11', '小行星', '中國', '時']


In [8]:
vec = model.wv['數學家']
print(vec.shape)
vec 

(300,)


array([-2.3187249 ,  0.01615903, -1.9287746 , -1.3532702 ,  2.0199182 ,
       -0.6695111 , -1.1122613 ,  0.8779186 , -1.8652799 , -1.3066971 ,
       -0.4766009 ,  1.4137077 ,  0.57277536, -0.08464076, -1.9201062 ,
        0.70093197, -1.9009708 , -1.0594636 , -2.0958002 , -0.7651449 ,
       -2.5452335 , -0.9265208 , -0.8671445 ,  0.06142833, -0.82512   ,
       -0.26285926,  1.2750754 ,  1.7273247 ,  0.0280052 , -0.04842537,
       -0.8294002 , -0.9745456 ,  1.3239641 ,  0.89738506, -0.8455278 ,
        2.8672667 , -1.0797536 ,  3.526261  , -1.1574554 , -0.4385191 ,
       -2.2924526 , -4.1270447 , -1.3164153 ,  0.82559234, -0.57494384,
       -2.0428135 ,  2.0156    , -0.21082444,  1.9229635 , -0.36971295,
       -1.5027969 ,  1.4621071 , -1.2802663 ,  0.21305539,  0.4693426 ,
       -0.52149993, -2.2269216 , -1.7401706 ,  0.51544434,  2.2579443 ,
       -1.5485755 ,  1.8927777 , -0.52080435, -2.2128117 , -1.3925284 ,
        2.726877  ,  0.45157835,  0.4734782 ,  2.0502794 ,  1.61

In [9]:
word = "這肯定沒見過 "

# 若強行取值會報錯
try:
    vec = model.wv[word]
except KeyError as e:
    print(e)

"Key '這肯定沒見過 ' not present"


In [10]:
model.wv.most_similar("飲料", topn=10)

[('飲品', 0.8093394041061401),
 ('果汁', 0.7081009745597839),
 ('軟飲料', 0.6987029910087585),
 ('酒精類', 0.6691874265670776),
 ('含酒精', 0.6534177660942078),
 ('瓶裝', 0.6331033706665039),
 ('酒類', 0.6328833699226379),
 ('提神', 0.6264342069625854),
 ('酒水', 0.6196794509887695),
 ('罐裝', 0.6145046353340149)]

In [11]:
model.wv.most_similar("car")

[('truck', 0.6725255250930786),
 ('cab', 0.6530406475067139),
 ('seat', 0.6503428220748901),
 ('tikita', 0.64206862449646),
 ('motor', 0.634936511516571),
 ('wagon', 0.6274935603141785),
 ('vehicle', 0.6248626708984375),
 ('driving', 0.623626708984375),
 ('cabriolet', 0.6228964328765869),
 ('luxury', 0.6170464754104614)]

In [12]:
model.wv.most_similar("facebook")

[('臉書', 0.8149164319038391),
 ('專頁', 0.7451366782188416),
 ('instagram', 0.7248266935348511),
 ('面書', 0.70846027135849),
 ('貼文', 0.7016114592552185),
 ('twitter', 0.6950445175170898),
 ('推特', 0.6841502785682678),
 ('粉絲團', 0.6587467193603516),
 ('粉專', 0.6559129357337952),
 ('tumblr', 0.6437175869941711)]

In [13]:
model.wv.most_similar("詐欺")

[('欺詐', 0.7152416110038757),
 ('詐騙', 0.6116501688957214),
 ('竊盜', 0.5657426118850708),
 ('慣犯', 0.5543817281723022),
 ('詐欺罪', 0.53859943151474),
 ('金光黨', 0.5374152660369873),
 ('信用調查', 0.5151287913322449),
 ('師篇', 0.5096631646156311),
 ('騙徒', 0.5079110860824585),
 ('前科', 0.5054307579994202)]

In [14]:
model.wv.most_similar("合約")

[('合同', 0.7782608270645142),
 ('簽約', 0.7128676772117615),
 ('續約', 0.6848387122154236),
 ('續簽', 0.6119592785835266),
 ('租約', 0.6113901734352112),
 ('選擇權', 0.5982978343963623),
 ('簽下', 0.597930908203125),
 ('買斷', 0.5780441164970398),
 ('解約', 0.5734624862670898),
 ('短約', 0.5693488717079163)]

In [15]:
model.wv.similarity("連結", "鍵接")

0.7164953

In [16]:
model.wv.similarity("連結", "陰天")

0.022799488

In [17]:
print(f"Loading {output_model}...")
new_model = word2vec.Word2Vec.load(output_model)

Loading word2vec.zh.300.model...


In [18]:
model.wv.similarity("連結", "陰天") == new_model.wv.similarity("連結", "陰天")

True